In [ ]:
import pandas as pd
from Bio import Entrez, Medline
import glob
import requests
import re
import os
from collections import defaultdict
import tarfile
import xml.etree.ElementTree as ElementTree
from bs4 import BeautifulSoup

In [ ]:
Entrez.api_key = ""
Entrez.email = ""

In [ ]:
def create_folders(folder_list):
    for i in folder_list:
        if not os.path.exists("../articles/"+i):
            os.makedirs("../articles/"+i)

In [ ]:
folder_list = ["zipped_documents", "unzipped_documents", "xml_files"]
create_folders(folder_list)

In [ ]:
def retrieve_pmc_list(query,db,amount):
    pubmed_id_list = []
    
    # retrieve a dictionary with metadata of articles regarding the query
    
    handle = Entrez.esearch(db="pmc", term=query, retmax=amount)
    record = Entrez.read(handle)
    # pulling out id's from the dictionary
    pubmed_id_list.append(record["IdList"])

    pmc_list = [item for sublist in pubmed_id_list for item in sublist]
   
    return pmc_list

In [ ]:
pmc_list = retrieve_pmc_list("rabies","pubmed","100")

In [ ]:
def retrieving_ftp_links(pmc_list):
    liste_ftp_links = []

    #retrieving ftp-link for every pmc id
    for i in pmc_list:
        page2 = requests.get('https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id='+i)
        page = requests.get('https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id='+i).content
        print(page2)
        soup = BeautifulSoup(page, 'html.parser')
        if "</error>" in soup:
            continue
        else:
            # catch only links that are zip files
            for link in soup.find_all('link'):
                if link.get('href').endswith('tar.gz'):
                    ftp_link = link.get('href')
                    liste_ftp_links.append(ftp_link[6:])
        
    return liste_ftp_links

In [ ]:
ftp_lst = retrieving_ftp_links(pmc_list)

In [ ]:
def downloading_fulltext(liste_ftp_links):
    for i in liste_ftp_links:
        path = "../articles/zipped_documents/" + i[-17:]
        response = requests.get("http://"+ i, stream=True)
        if response.status_code == 200:
            with open(path, 'wb') as f:
                f.write(response.raw.read())

In [ ]:
downloading_fulltext(ftp_lst)

In [ ]:
liste_zipped_files = glob.glob("../articles/zipped_documents/*")

In [ ]:
def extract_files(liste_zipped_files):
    for i in liste_zipped_files:
        if i.endswith("tar.gz"):
            tar = tarfile.open(i, "r:gz")
            tar.extractall("../articles/unzipped_documents")
            tar.close()
        elif i.endswith("tar"):
            tar = tarfile.open(i, "r:")
            tar.extractall()
            tar.close()

In [ ]:
extract_files(liste_zipped_files)

In [ ]:
def rename_files():
    liste_xml = glob.glob("../articles/unzipped_documents/*/*.nxml")

    for i in liste_xml:
        m = re.search("PMC\d+", i).group(0)
        #print(m)

        os.rename(i, "../articles/xml_files/" + m + '.nxml')

In [ ]:
rename_files()